In [142]:
import scipy.io as sio
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from scipy.stats.stats import pearsonr as ps
import math

In [143]:
# load data
nP = 9 
nTrial = 360
nWord = 60
nRep = 6
nFeature = 25

contents = []
data = []
info = []
meta = []
for i in range(nP):
    contents.append(sio.loadmat('./fMRI_data/data-science-P'+str(i+1)+'.mat'))
    data.append(contents[i]['data'])
    info.append(contents[i]['info'])
    meta.append(contents[i]['meta'])

In [144]:
X = np.array(pd.read_excel('./fMRI_data/features4.xls'))
X = X.T
X = X * np.divide(1, np.linalg.norm(X, axis = 1))[:, np.newaxis]

In [147]:
words = ['airplane', 'ant', 'apartment', 'arch', 'arm', 'barn', 'bear', 'bed', 'bee', 'beetle', \
         'bell', 'bicycle', 'bottle', 'butterfly', 'car', 'carrot', 'cat', 'celery', 'chair', \
         'chimney', 'chisel', 'church', 'closet', 'coat', 'corn', 'cow', 'cup', 'desk', 'dog', \
         'door', 'dress', 'dresser', 'eye', 'fly', 'foot', 'glass', 'hammer', 'hand', 'horse', \
         'house', 'igloo', 'key', 'knife', 'leg', 'lettuce', 'pants', 'pliers', 'refrigerator', \
         'saw', 'screwdriver', 'shirt', 'skirt', 'spoon', 'table', 'telephone', 'tomato', \
         'train', 'truck', 'watch', 'window']

# Classify same words
word_index = [[] for k in range(nWord)]
for word in words:
    for j in range(nTrial):
        if info[0][0][j][2][0] == word:
            word_index[words.index(word)].append(j)
            
counts = []
for i in range(nP-1,0,-1):  
    nVoxel = len(data[i][0][0][0])
    Y = np.zeros((nWord, nVoxel)) # 60 images for training
    Y_cub = np.zeros((nWord, nRep, nVoxel)) # 60*6 images for choosing voxels
    count = 0
    print(i)
    # Average same words
    for k in range(nWord):
        for m in range(nRep):
            Y[k] += data[i][word_index[k][m]][0][0]
            Y_cub[k][m]=data[i][word_index[k][m]][0][0]
        Y[k] /= nRep
    
    
    # Begin bootstrap
    for k in range(0,nWord-1):
        for kk in range(k+1, nWord):
            X_tr = np.concatenate((X[0:kk],X[kk+1:k],X[k+1:nWord]), axis = 0)
            Y_tr = np.concatenate((Y[0:kk],Y[kk+1:k],Y[k+1:nWord]), axis = 0)
            mu_x = np.mean(X_tr, axis = 0)
            mu_y = np.mean(Y_tr, axis = 0)
            X_tr -= mu_x
            Y_tr -= mu_y
            X_va = X[[kk, k]] - mu_x
            Y_va = Y[[kk, k]] - mu_y
            W = np.linalg.solve(X_tr.T@X_tr, X_tr.T@Y_tr)
            Y_pred = X_va@W
            
            
            # Cosine similarity voxel selection
            corre = []
            Y_cos = np.concatenate((Y_cub[0:k],Y_cub[k+1:kk],Y_cub[kk+1:]),axis=0)
            for ivox in range(nVoxel):
                Y_ivox = Y_cos[:,:,ivox]
                cor_count = 0;
                for irep in range(nRep-1):
                    for iirep in range(irep+1, nRep):
                        cor_count += ps(Y_ivox[:,irep],Y_ivox[:,iirep])[0]
                corre.append(cor_count/15)
            ind = np.argsort(np.array(corre))[-500:]
            
            match1122 = cosine_similarity(Y_pred[0][ind].reshape(1,-1), Y_va[0][ind].reshape(1,-1)) + \
                        cosine_similarity(Y_pred[1][ind].reshape(1,-1), Y_va[1][ind].reshape(1,-1))
            match1221 = cosine_similarity(Y_pred[0][ind].reshape(1,-1), Y_va[1][ind].reshape(1,-1)) + \
                        cosine_similarity(Y_pred[1][ind].reshape(1,-1), Y_va[0][ind].reshape(1,-1))
            
            '''
            match1122 = cosine_similarity(Y_pred[0].reshape(1,-1), Y_va[0].reshape(1,-1)) + \
                        cosine_similarity(Y_pred[1].reshape(1,-1), Y_va[1].reshape(1,-1))
            match1221 = cosine_similarity(Y_pred[0].reshape(1,-1), Y_va[1].reshape(1,-1)) + \
                        cosine_similarity(Y_pred[1].reshape(1,-1), Y_va[0].reshape(1,-1))
            '''
            
            #print(float(match1122), float(match1221))
            if match1122 > match1221: count += 1    
    counts.append(count/(nWord-1))
    print(counts)

8
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
[0.5084745762711864]
7
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
[0.5084745762711864, 0.576271186440678]
6
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
[0.5084745762711864, 0.576271186440678, 0.7457627118644068]
5
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
[0.5084745762711864, 0.576271186440678, 0.7457627118644068, 0.6271186440677966]
4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [150]:
counts

[0.5084745762711864,
 0.576271186440678,
 0.7457627118644068,
 0.6271186440677966,
 0.711864406779661,
 0.8135593220338984,
 0.7966101694915254,
 0.8305084745762712]

In [151]:
np.mean(counts)

0.701271186440678